### NLP using Transformers
- We will be looking at how to use transformer models obtained from the Hugging Face website to perform Natural Language Processing tasks.
- Text classification

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [43]:
from datasets import load_dataset

imdb = load_dataset("imdb")

In [44]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [45]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [46]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [47]:
# apply the preprocessing function over the entire dataset
tokenized_imdb = imdb.map(preprocess_function, batched=True) # batched = True divided datasets into batches and preprocesses them at once

Map: 100%|██████████| 50000/50000 [00:36<00:00, 1368.89 examples/s]


In [48]:
from transformers import DataCollatorWithPadding

# padding is the process of adding special tokens or symbols to input sequences to make them of equal length

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [49]:
import evaluate

accuracy = evaluate.load("accuracy")

In [50]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [51]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [59]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_imdb["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

ImportError: 
create_optimizer requires the TensorFlow library but it was not found in your environment. Checkout the instructions on the
installation page: https://www.tensorflow.org/install and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)